In [1]:
import os
import numpy as np
from astropy.io import fits
import astropy.units as u
from astropy.wcs import WCS

from astropy.wcs.utils import proj_plane_pixel_scales

In [2]:
# read in the M101 image
# hdu_in = fits.open("data/m101_mips24_26mar07.cal.fits")
hdu_in = fits.open("data/m101_irac4_19apr06.cal.fits")
# boost the IRAC4 data to make it easy to see with MIRI (smaller pixels and not that many wavelengths)
data_in = hdu_in[0].data * 100.0   # for IRAC 8
# data_in = hdu_in[1].data  # for MIPS 24
wcs_in = w = WCS(hdu_in[0].header)

In [3]:
# setup the output data cube info
n_waves = 2
dwave = 0.1
start_wave = 7.6
odpix = np.average(proj_plane_pixel_scales(wcs_in) * 3600.)
print(odpix)
dpix = odpix * (0.8 / 6.5) # * (7.8 / 24.0)
print(dpix)

0.6099984
0.07507672615384617


In [4]:
# populate the output data with the input data
naxis_in = data_in.shape
naxis = (naxis_in[0], naxis_in[1], n_waves)

data = np.zeros(naxis)
for k in range(n_waves):
    # convert to Jy/sr
    data[:, :, k] = data_in * 1e6

In [5]:
# Index of the reference pixel
crpix = (1, 1, 1)

# Starts at 1

# Value of the reference pixel
# Axis 1 & 2 are in ra,dec ; Axis 3 is in micron
crval = (0., 0., start_wave)
cdelt = (dpix, dpix, dwave)

# Prepare data to correct dimension order
data = np.moveaxis(data, -1, 0)
data = np.flip(data, axis=1)
hdu = fits.PrimaryHDU(data)
hdu.header['CRVAL1'] = crval[0]
hdu.header['CRPIX1'] = crpix[0]
hdu.header['CDELT1'] = cdelt[0]
hdu.header['CTYPE1'] = "RA---TAN"
hdu.header['CUNIT1'] = u.arcsec.to_string()

hdu.header['CRVAL2'] = crval[1]
hdu.header['CRPIX2'] = crpix[1]
hdu.header['CDELT2'] = cdelt[1]
hdu.header['CTYPE2'] = "DEC--TAN"
hdu.header['CUNIT2'] = u.arcsec.to_string()

hdu.header['CRVAL3'] = crval[2]
hdu.header['CRPIX3'] = crpix[2]
hdu.header['CDELT3'] = cdelt[2]
hdu.header['CTYPE3'] = "WAVE"
hdu.header['CUNIT3'] = u.micron.to_string()

hdu.header['UNITS'] = (u.Jy / u.sr).to_string()

outFileName = 'scenes/dither_illustration_m101_irac8_SED.fits'
if os.path.isfile(outFileName):
    os.remove(outFileName)
hdu.writeto(outFileName)